In [1]:
import os
import warnings

import dask
import ncar_jobqueue
import yaml

import utils

%matplotlib inline

In [2]:
with open("diag_metadata.yaml", mode="r") as fptr:
    diag_metadata_list = yaml.safe_load(fptr)

# varnames = utils.get_varnames_from_metadata_list(diag_metadata_list)

In [3]:
def summary_plots(ds, diag_metadata, save_pngs=False):
    varname = diag_metadata["varname"]
    print(varname)
    da = ds[varname].isel(diag_metadata.get("isel_dict"))

    utils.summary_plot_global_ts(
        ds,
        da,
        diag_metadata,
        time_coarsen_len=12,
        save_pngs=save_pngs,
        casename=ds.attrs["title"],
        savefig_kwargs={"dpi": 72},  # match default behavior of savefig
    )

    utils.summary_plot_histogram(
        ds,
        da,
        diag_metadata,
        save_pngs=save_pngs,
        casename=ds.attrs["title"],
        savefig_kwargs={"dpi": 72},  # match default behavior of savefig
    )

In [4]:
from dask_jobqueue import PBSCluster
from dask.distributed import Client

ncores = 2
nmem = "64GB"

cluster = PBSCluster(
    cores=ncores,  # The number of cores you want
    memory=nmem,  # Amount of memory
    processes=ncores,  # How many processes
    queue="casper",  # The type of queue to utilize (/glade/u/apps/dav/opt/usr/bin/execcasper)
    resource_spec="select=1:ncpus="
    + str(ncores)
    + ":mem="
    + nmem,  # Specify resources
    project="NCGD0011",  # Input your project ID here
    walltime="6:00:00",  # Amount of wall time
    interface="ib0",  # Interface to use
)

cluster.scale(8)
print(f"dashboard_link={cluster.dashboard_link}")

dashboard_link=https://jupyterhub.ucar.edu/dav/user/mlevy/proxy/8787/status


In [5]:
casename = "g.e22.G1850ECO_JRA_HR.TL319_t13.004"
campaign_root = os.path.join(
    os.sep,
    "glade",
    "campaign",
    "cesm",
    "development",
    "bgcwg",
    "projects",
    "hi-res_JRA",
    "cases",
)

# Set up CaseClass object
case = utils.CaseClass(
    casename, os.path.join(campaign_root, casename, "output")
)

In [6]:
with dask.distributed.Client(cluster) as client:
    stream = "pop.h"
    #     ds = case.gen_dataset(varnames, stream)
    for diag_metadata in diag_metadata_list:
        ds = case.gen_dataset(diag_metadata["varname"], stream, end_year=17)
        summary_plots(ds, diag_metadata, save_pngs=True)

Datasets contain a total of 204 time samples
Last average written at 0018-01-01 00:00:00
POC_FLUX_100m
Datasets contain a total of 204 time samples
Last average written at 0018-01-01 00:00:00
CaCO3_FLUX_100m
Datasets contain a total of 204 time samples
Last average written at 0018-01-01 00:00:00
FG_CO2
Datasets contain a total of 204 time samples
Last average written at 0018-01-01 00:00:00
DpCO2
Datasets contain a total of 204 time samples
Last average written at 0018-01-01 00:00:00
PH
Datasets contain a total of 204 time samples
Last average written at 0018-01-01 00:00:00
spChl
Datasets contain a total of 204 time samples
Last average written at 0018-01-01 00:00:00
photoC_sp_zint
Datasets contain a total of 204 time samples
Last average written at 0018-01-01 00:00:00
coccoChl
Datasets contain a total of 204 time samples
Last average written at 0018-01-01 00:00:00
photoC_cocco_zint
Datasets contain a total of 204 time samples
Last average written at 0018-01-01 00:00:00
diatChl
Datasets

In [7]:
cluster.close()